In [1]:
#we may need some code in the ../python directory and/or matplotlib styles
import sys
import matplotlib as mpl
sys.path.append('../python/')
os.environ['MPLCONFIGDIR'] = '../mplstyles'


#fonts
# Set the font dictionaries (for plot title and axis titles)
title_font = {'fontname':'Arial', 'size':'16', 'color':'black', 'weight':'normal',
              'verticalalignment':'bottom'} # Bottom vertical alignment for more space
axis_font = {'fontname':'Arial', 'size':'32'}
legend_font = {'fontname':'Arial', 'size':'22'}

#fonts global settings
mpl.rc('font',family=legend_font['fontname'])

import numpy as np
#set up a plot 
from matplotlib import pyplot as plt
#got smarter about the mpl config: see mplstyles/ directory
plt.style.use('standard')

# Joint Distributions of $E_p$ and $E_q$ with SuperCDMS Goal Resolutions

The purpose of this notebook is to detail the calculation of the joint distribution of $E_p$ and $E_q$ (the phonon and ionization energies) for SuperCDMS iZIP detectors with the SuperCDMS SNOLAB goal resolutions. Specifically, we are interested in at which energy the electron-recoil (ER) and nuclear-recoil (NR) bands overlap for a given effective nuclear-recoil Fano factor. The lower this energy is the more low-mass reach the iZIP SuperCDMS search will have in an essentially background-free environment. 

Like the notebook `QEr_2D_joint.ipynb` we will build up the distributions from conditional probability arguments beginning with the random variables and constants in the following table. 

variable/constant name|Description 
:-|:-
Y|measured ionization yield 
N|number of e/h pairs
$\delta Q$ | instrumental fluctuation on the ionization measurement
$\delta P$ | instrumental fluctuation on the phonon measurement
$E_r$ | true recoil energy
$\epsilon$ | average energy to create a single electron-hole pair for an electron-recoil in keV
$V$ |voltage across the detector in Volts
F | effective Fano factor for nuclear-recoils

We seek the joint distribution of $E_p$ and $E_q$ given a fixed true energy, $E_r$. It is equal to the 5-variable joint distribution integrated appropriately over the 3 unwanted variables. 

\begin{equation}
P(E_p,E_q | E_r) = \int_{-\infty}^{\infty} d(\delta P) \int_{-\infty}^{\infty} d(\delta Q) \int_0^{\infty} dN
 P(E_p,E_q,\delta P,\delta Q, N | E_r)
\end{equation}

The technique used is to write down the 5-variable joint distribution based on the rules of conditional probability and the fact that N, $\delta P$, and $\delta Q$ are independent random variables. Then the appropriate integrals can be carried out analytically or numerically.

## Resolutions

In addition to the variables already defined we need some kind of specifications for the (assumed Gaussian) resolutions on the three fundamental measured variables: N, $\delta Q$, and $\delta P$. The resolutions are denoted $\sigma_N$, $\sigma_q$, and $\sigma_p$ respectively, and are assumed to be functions of the true recoil energy $E_r$.

Both the $\sigma_q(E_r)$ and the $\sigma_p(E_r)$ are composed of the baseline resolutions in the SuperCDMS SNOLAB sensitivity paper [REF] and reasonable guesses for the energy-dependent reconstruction resolution. 

The $\sigma_N$ is the standard deviation of the number of electron-hole pairs, which can be computed (for a fixed $E_r$) using the effective Fano factor F. It is given simply by:

\begin{equation}
\sigma_N(E_r) = \sqrt{F(\bar{Y}E_r/\epsilon)},
\end{equation}

Where $\bar{Y}$ is the mean ionization yield at $E_r$. Whenever $\bar{Y}$ is written it is tacitly assumed to be a function of the true recoil energy, $E_r$.

In [5]:
#constants
V=4.0
eps=3.3/1000.0
F=0.00001

#get baseline resolutions for SuperCDMS goal for iZIPs
sig0p = 0.050 #50 eV
sig0q = 0.100 #100 eV


#sigp and sigq functions for ERs
sigp = lambda E: np.sqrt(sig0p**2 + 0.1*E)
sigq = lambda E: np.sqrt(sig0q**2 + 0.1*E)

#get the 2D resolution function
import prob_dist as pd
P = pd.EpEq_v2_2D_fast(sigp,sigq,V,eps) #currently using Edw ionization yield model (not Lindhard)

#test for reasonable pars
Er0 = 5
Eq0 = (0.16*Er0**0.18)*Er0
Ep0 = Er0 + (V/(1000*eps))*Eq0
print(P(Ep0,Eq0,Er0))

0.01174313789596759
